In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
%matplotlib inline
import nltk
from IPython.display import Image

In [ ]:
nltk.download()

In [17]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

## Classificador de spam usando python

Este notebook mostrará frutos dos meus estudos sobre NLP usando python. Nele combinaremos técnicas de Machine Learning, processamento de texto e estatística para obter os textos em um formato que o algoritmo de aprendizado usado possa entender. 

### Entendendo os dados

O conjunto de dados desse notebook pertence a UCI. O arquivo tem mais de 5 mil mensagens rotuladas como HAM ou SPAM. 

Primeiramente iremos obter todas as mensagens em uma variável.

In [5]:
mensagens = [line.rstrip() for line in open('smsspamcollection/SMSSpamCollection')]

#### O arquivo é separado por tabulação

Usaremos o pandas para não ter que analisar os arquivos manualmente

In [6]:
mensagens = pd.read_csv('smsspamcollection/SMSSpamCollection', sep='\t',
                           names=["tipo", "mensagem"])
mensagens.head()

,tipo,mensagem
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Preparando o texto para o modelo

O principal problema a ser enfrentado aqui é que os dados são do tipo String e o algoritmo que será utilizado necessita de um vetor numérico para fazer a classificação. 

Um dos métodos mais simples para converter o texto em um vetor é o Bag of Words, sua tradução livre seria "Saco de palavras".  Nele cada palavra única é representada por um número. 

O primeiro passo para preparar o texto é remover as pontuações e as chamadas stopwords de cada mensagem do nosso dataframe. 

In [7]:
stopwords.words('english')[0:10] # Alguns exemplos de stopwords

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

Para isso, criaremos a seguinte função:

In [8]:
def preProcessamento(msg):
    # Retira pontuações
    noPont = [char for char in msg if char not in string.punctuation]

    # Junta-os para formar strings
    noPont = ''.join(noPont)
    
    # Remove as stopwords
    return [word for word in noPont.split() if word.lower() not in stopwords.words('english')]

In [9]:
mensagens['mensagem'].head(5).apply(preProcessamento)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: mensagem, dtype: object

Podemos verificar que nossa função funciona e retorna apenas as palavras que realmente nos importam. 

### Transformando as mensagens em um vetor

Vamos fazer isso em três etapas usando o modelo Bag of Words:

    1. Contar quantas vezes ocorre uma palavra em cada mensagem (conhecida como frequência de termo)

    2. Pesar as contagens, de modo que as palavras frequentes recebem menor peso (frequência inversa do documento)

    3. Normalize os vetores para o comprimento da unidade, para abstrair do comprimento do texto original (norma L2)


É possível notar que cada vetor terá **n** dimensões onde **n** = **quantidade de palavras únicas na mensagem**. 

Usaremos o CountVectorizer do SciKit Learn. Este modelo converterá uma coleção de documentos de texto em uma matriz de contagem de palavras relevantes no nosso texto. 

O resultado será uma matriz bidimensional onde as colunas são representadas pelas mensagens e as linhas a contagem de palavras. 

Por exemplo: 

![title](table.png)			

Há muitos argumentos e parâmetros que podem ser passados para o CountVectorizer. Neste caso, vamos especificar o analyzer para ser nossa própria função previamente definida:

In [10]:
# Isso pode demorar um pouco
bow_transformer = CountVectorizer(analyzer=preProcessamento).fit(mensagens['mensagem'])

print(len(bow_transformer.vocabulary_))

11425


Agora iremos usar nosso objeto 'bow_transformer' para transformar todo o dataframe de mensagens em suas representações vetoriais:

In [11]:
messages_bow = bow_transformer.transform(mensagens['mensagem'])

Após a contagem, o termo ponderação e normalização pode ser feito com TF-IDF, usando o TfidfTransformer do scikit-learn.

### TF - IDF

O valor TF-IDF, é uma medida estatística que tem o intuito de indicar a importância de uma palavra de um documento em relação a uma coleção de documentos ou em um corpus linguístico. Ela é frequentemente utilizada como fator de ponderação na recuperação de informações e na mineração de dados.

O valor TF-IDF de uma palavra aumenta proporcionalmente à medida que aumenta o número de ocorrências dela em um documento, no entanto, esse valor é equilibrado pela frequência da palavra no corpus. Isso auxilia a distinguir o fato da ocorrência de algumas palavras serem geralmente mais comuns que outras. 

Normalmente, o peso de TF-IDF é composto por dois termos: o primeiro calcula a Freqüência do termo normalizada (TF), ou seja, o número de vezes que uma palavra aparece em um documento, dividido pelo número total de palavras nesse documento; O segundo termo é a Freqüência do Documento Inverso (IDF), calculado como o logaritmo do número de documentos no corpus dividido pela quantidade de documentos onde o termo específico aparece.

TF(x) = número de vezes que X aparece em um documento / número total de termos do documento

IDF(x) = log_e (Número total de documentos / Número de documentos com termo X nele)

**Exemplo:**

Seja B um conjunto com 1000 documentos e A um documento que contém 100 palavras. Supondo que a palavra "java" aparece 3 vezes no documento A e que em 10 dos 1000 documentos aparece a palavra "java", temos: 

O TF(x) para x = "java" é igual a 3/100. Isto é 0,03.

O IDF(x) do documento é igual log(1000/10). Isto é 2.


Agora faremos isso no SciKit Learn:

In [12]:
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)

### Treinando o modelo e fazendo predições

Com mensagens representadas como vetores, podemos finalmente treinar nosso classificador de spam. Agora, podemos realmente usar quase qualquer tipo de algoritmos de classificação. Para esse modelo o algoritmo do classificador Naive Bayes é uma boa escolha.

Primeiramente iremos dividir nossos dados em dados de treino e dados de teste.

Vamos executar o nosso modelo novamente e depois prever o conjunto de testes. Usaremos os recursos pipeline do SciKit Learn para armazenar uma linha de fluxo de trabalho. Isso nos permitirá configurar todas as transformações que faremos aos dados para uso futuro. Vejamos um exemplo de como funciona:

In [13]:
msg_train, msg_test, tipo_train, tipo_test = \
train_test_split(mensagens['mensagem'], mensagens['tipo'], test_size=0.2)

print(len(msg_train), len(msg_test), len(msg_train) + len(msg_test))

4457 1115 5572


In [19]:
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=preProcessamento)),  # Tokeniza as mensagens
    ('tfidf', TfidfTransformer()),  # Faz a transformação em TF-IDF
    ('classifier', MultinomialNB()),  # Define a classe que realizará nossa classificação.
])

In [20]:
pipeline.fit(msg_train,tipo_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function preProcessamento at 0x7f1ff6387440>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [22]:
predictions = pipeline.predict(msg_test)

### Resultado final

In [24]:
print(classification_report(predictions,tipo_test))

              precision    recall  f1-score   support

         ham       1.00      0.96      0.98      1013
        spam       0.71      1.00      0.83       102

    accuracy                           0.96      1115
   macro avg       0.85      0.98      0.90      1115
weighted avg       0.97      0.96      0.97      1115



Podemos inferir do que foi printado que tivemos um excelente modelo e um bom resultado.